In [1]:
import ee
import folium
import geehydro
import geemap

C:\Users\srini\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
ee.Initialize(project='ml-meets-the-sun')

In [3]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [4]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [5]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [6]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [7]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [8]:
AG100_elev = ee.Image('NASA/ASTER_GED/AG100_003');
elevation = AG100_elev.select('elevation');
elevationVis = {
  'min': -15.0,
  'max': 5000.0,
  'palette' : [
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef', '3ae237',
    'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08',
    'ff500d', 'ff0000', 'de0101', 'c21301'
  ],
}
Map.setCenter(15.3173,75.7139)
Map.addLayer(elevation, elevationVis, 'Elevation')

In [22]:
AG100_elev.projection()

In [9]:
result_clip = {}

def elev_clip(state, name):
    cname = name + "_elev"
    clipped = elevation.clip(state)
    result_clip[cname] = clipped

In [11]:
for name,state in states.items():
   elev_clip(state, name)

In [12]:
result_clip

{'Karnataka_elev': <ee.image.Image at 0x2729e4c7f70>,
 'Tamil Nadu_elev': <ee.image.Image at 0x2729e4c6620>,
 'Rajasthan_elev': <ee.image.Image at 0x2729e4c5330>}

In [13]:
def display_elev(cname, clipped):
    
    elevationVis = {
    'min': -15.0,
    'max': 5000.0,
    'palette' : [
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef', '3ae237',
    'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08',
    'ff500d', 'ff0000', 'de0101', 'c21301'
    ],
    }
   
    Map.addLayer(clipped, elevationVis, cname, True, 0.5)
    return Map

In [14]:
for cname,clipped in result_clip.items():
    display_elev(cname, clipped)

In [15]:
result_clip['Karnataka_elev'].projection()

In [16]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_elev'],
    description='Karnataka_elev',
    assetId='projects/ml-meets-the-sun/assets/KA_elev',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [17]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_elev'],
    description='Tamil Nadu_elev',
    assetId='projects/ml-meets-the-sun/assets/TN_elev',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [18]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_elev'],
    description='Rajasthan_elev',
    assetId='projects/ml-meets-the-sun/assets/RA_elev',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [19]:
karnataka_elev = ee.Image('projects/ml-meets-the-sun/assets/KA_elev')
TamilNadu_elev = ee.Image('projects/ml-meets-the-sun/assets/TN_elev')
Rajasthan_elev = ee.Image('projects/ml-meets-the-sun/assets/RA_elev')

In [26]:
Map.addLayer(karnataka_elev, elevationVis, 'ERA5_KA_elev', True, 0.5)
Map.addLayer(TamilNadu_elev, elevationVis, 'ERA5_TN_elev', True, 0.5)
Map.addLayer(Rajasthan_elev, elevationVis, 'ERA5_RA_elev', True, 0.5)

In [34]:
karnataka_elev.projection()

In [27]:
Map

Map(center=[75.7139, 15.3173], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [20]:
slope_KA = ee.Terrain.slope(karnataka_elev);
slope_TN = ee.Terrain.slope(TamilNadu_elev);
slope_RA = ee.Terrain.slope(Rajasthan_elev);



In [23]:
Map.addLayer(slope_KA, elevationVis, 'slope_KA', True, 0.5)
Map.addLayer(slope_TN, elevationVis, 'slope_TN', True, 0.5)
Map.addLayer(slope_RA, elevationVis, 'slope_RA', True, 0.5)

In [37]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=slope_KA,
    description='slope_KA',
    assetId='projects/ml-meets-the-sun/assets/slope_KA',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [ ]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=slope_TN,
    description='slope_TN',
    assetId='projects/ml-meets-the-sun/assets/slope_TN',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [24]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=slope_RA,
    description='slope_RA',
    assetId='projects/ml-meets-the-sun/assets/slope_RA',
    region=region,
    crsTransform = [0.001, 0, -180, 0, -0.001, 80],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()